## Purpose

Reference for adding SOHO position information to LASCO C3 Helioviewer images: https://docs.sunpy.org/en/latest/generated/gallery/units_and_coordinates/getting_lasco_observer_location.html.

### HEC coordinates

From From https://soho.nascom.nasa.gov/data/ancillary/. See SPACECRAFT ORBIT COORDINATES.

The Heliocentric Ecliptic coordinate system is defined as follows: the origin is Sun centered, with the Z axis parallel to the ecliptic pole with positive north of the ecliptic plane; the X-Y plane lies in the ecliptic plane and the X axis points towards the first point of Aries; the Y axis completes a right-handed orthogonal coordinate system.


### Heliocentric Aries Ecliptic (Mean) coordinates (`SunPy` HAE or HEC, see https://docs.sunpy.org/en/stable/reference/coordinates/index.html)

From https://docs.astropy.org/en/stable/api/astropy.coordinates.HeliocentricMeanEcliptic.html#astropy.coordinates.HeliocentricMeanEcliptic.

Heliocentric mean ecliptic coordinates. These origin of the coordinates are the center of the sun, with the x axis pointing in the direction of the mean (not true) equinox as at the time specified by the equinox attribute (as seen from Earth), and the xy-plane in the plane of the ecliptic for that date.


### GCI coordinates 

From https://soho.nascom.nasa.gov/data/ancillary/. See SPACECRAFT ORBIT COORDINATES.

The GCI coordinate system is defined as follows: Earth centered, where the X axis points from the Earth towards the first point of Aries (the position of the Sun at the vernal equinox). This direction is the intersection of the Earth's equatorial plane and the ecliptic plane --- thus the X axis lies in both planes. The Z axis is parallel to the rotation axis of the Earth and the Y axis completes a right-handed orthogonal coordinate system. As mentioned above, the X axis is the direction of the mean vernal equinox of J2000. The Z axis is also defined as being normal to the mean Earth equator of J2000.


### Geocentric Earth Equatorial coordinates (GEI in `SunPy`)

From https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.GeocentricEarthEquatorial.html#sunpy.coordinates.frames.GeocentricEarthEquatorial.

A coordinate or frame in the Geocentric Earth Equatorial (GEI) system.

- The origin is the center of the Earth.
- The Z-axis (+90 degrees latitude) is aligned with the Earth’s north pole.
- The X-axis (0 degrees longitude and 0 degrees latitude) is aligned with the mean (not true) vernal equinox.


### Heliographic Carrington coordinates (HGC in `SunPy`)

From https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.HeliographicCarrington.html#sunpy.coordinates.frames.HeliographicCarrington.

A coordinate or frame in the Carrington Heliographic (HGC) system.
- The origin is the center of the Sun.
- The Z-axis (+90 degrees latitude) is aligned with the Sun’s north pole.
- The X-axis and Y-axis rotate with a period of 25.38 days.

This system differs from Stonyhurst Heliographic (HGS) in its definition of longitude. This longitude is an “apparent” longitude because it takes into account the time it takes for light to travel from the Sun’s surface to the observer (see Calculating Carrington longitude). Thus, the observer needs to be specified to be able to transform to any other coordinate frame.


### Heliographic Stonyhurst coordinates (`HGS` in `SunPy`)

A coordinate or frame in the Stonyhurst Heliographic (HGS) system.
- The origin is the center of the Sun.
- The Z-axis (+90 degrees latitude) is aligned with the Sun’s north pole.
- The X-axis (0 degrees longitude and 0 degrees latitude) is aligned with the projection of the Sun-Earth line onto the Sun’s equatorial plane.

This system is also know as the Heliocentric Earth Equatorial (HEEQ) system when represented using Cartesian components.

In [1]:
import dataclasses
from dataclasses import dataclass
from datetime import datetime

import numpy as np
import astropy.units as u
from astropy.coordinates import HeliocentricMeanEcliptic, HCRS
from sunpy.coordinates.frames import GeocentricEarthEquatorial, GeocentricSolarEcliptic, GeocentricSolarMagnetospheric, HeliographicCarrington, HeliographicStonyhurst
from sunpy.coordinates.ephemeris import get_horizons_coord

In [2]:
# Extracted from SO_OR_PRE_20080201_V04.DAT

DAT_LINE = "01-Feb-2008 20:50:00.000 2008  32 75000000     1337759.22141    -824922.22264    -479016.13027   0.0838840   0.0742209   0.0061811    1600807.58574     352939.24964    -111363.46095   0.0765737  -0.2187542  -0.0238535    1600807.58574     366972.47487      47949.31764   0.0765737  -0.5501509   2.6556334   99132463.14094 -100093435.80465  -43394334.38088  -97794703.91953  108147813.77940    -112712.32332 -22.4595017 -20.0711358  -0.0237128 2066   4.564 -0.105 2066   4.562 -0.106"
# DAT_LINE = "09-May-2008 20:50:00.000 2008 130 75000000      959534.59929     744462.20031     432054.46248  -0.3899428   0.3233711   0.1572567    1273307.05270    -173876.74223     100288.09260  -0.0142633   0.2811878   0.0156643    1273307.05270    -200055.30662      16391.98893  -0.0142633   0.1516806  -1.0357047   98157853.30651  105352743.94729   45673696.82973  -97198318.70722 -113972313.87727     102389.46974  21.7794013 -19.1048280   0.0168796 2069   0.856 -0.056 2069   0.857 -0.055"
# DAT_LINE = "23-Mar-2008 15:00:00.000 2008  83 54000000     1513698.98474    -464210.01966    -271168.20176   0.0266075   0.1582179   0.1126081    1481059.99203    -618590.91774     -64143.03467  -0.0865446  -0.1084111   0.0403897    1481059.99203    -566288.52246    -257072.67793  -0.0865446   2.8019642  -6.4229566  148893626.80979    7745041.16391    3356980.41582 -147379927.82505   -8975035.34234     -63308.68702   1.2224660 -29.6512619   0.0415401 2068   5.461 -0.121 2068   5.465 -0.121"

In [3]:
def __post_init__(self):
    # https://stackoverflow.com/questions/54863458/force-type-conversion-in-python-dataclass-init-method
    for field in dataclasses.fields(self):
        value = getattr(self, field.name)
        if not isinstance(value, field.type):
            setattr(self, field.name, field.type(value))
            # raise ValueError(f'Expected {field.name} to be {field.type}, '
                             # f'got {repr(value)}')


def autoconvert_dataclass(x):
    x.__post_init__ = __post_init__
    y = dataclass(x)
    return y

In [4]:
@autoconvert_dataclass
class DAT:
    date: str
    time: str
    year: int
    doy: int
    ellapsed_ms: int
    gci_x_km: float
    gci_y_km: float
    gci_z_km: float
    gci_vx_kms: float
    gci_vy_kms: float
    gci_vz_kms: float
    gse_x_km: float
    gse_y_km: float
    gse_z_km: float
    gse_vx_kms: float
    gse_vy_kms: float
    gse_vz_kms: float
    gsm_x_km: float
    gsm_y_km: float
    gsm_z_km: float
    gsm_vx_kms: float
    gsm_vy_kms: float
    gsm_vz_kms: float
    gci_sun_vector_x_km: float
    gci_sun_vector_y_km: float
    gci_sun_vector_z_km: float
    hec_x_km: float
    hec_y_km: float
    hec_z_km: float
    hec_vx_kms: float
    hec_vy_kms: float
    hec_vz_kms: float
    cr_earth: int
    heliographic_lon_earth: float
    heliographic_lat_earth: float
    cr_soho: int
    heliographic_lon_soho: float
    heliographic_lat_soho: float

In [5]:
dat = DAT(*DAT_LINE.split())
dat

DAT(date='01-Feb-2008', time='20:50:00.000', year=2008, doy=32, ellapsed_ms=75000000, gci_x_km=1337759.22141, gci_y_km=-824922.22264, gci_z_km=-479016.13027, gci_vx_kms=0.083884, gci_vy_kms=0.0742209, gci_vz_kms=0.0061811, gse_x_km=1600807.58574, gse_y_km=352939.24964, gse_z_km=-111363.46095, gse_vx_kms=0.0765737, gse_vy_kms=-0.2187542, gse_vz_kms=-0.0238535, gsm_x_km=1600807.58574, gsm_y_km=366972.47487, gsm_z_km=47949.31764, gsm_vx_kms=0.0765737, gsm_vy_kms=-0.5501509, gsm_vz_kms=2.6556334, gci_sun_vector_x_km=99132463.14094, gci_sun_vector_y_km=-100093435.80465, gci_sun_vector_z_km=-43394334.38088, hec_x_km=-97794703.91953, hec_y_km=108147813.7794, hec_z_km=-112712.32332, hec_vx_kms=-22.4595017, hec_vy_kms=-20.0711358, hec_vz_kms=-0.0237128, cr_earth=2066, heliographic_lon_earth=4.564, heliographic_lat_earth=-0.105, cr_soho=2066, heliographic_lon_soho=4.562, heliographic_lat_soho=-0.106)

In [6]:
DATE_OBS = datetime.strptime(dat.date, '%d-%b-%Y').strftime('%Y-%m-%d') + 'T' + dat.time
DATE_OBS

'2008-02-01T20:50:00.000'

In [7]:
soho_hae = HeliocentricMeanEcliptic(x=dat.hec_x_km * u.km, y=dat.hec_y_km * u.km, z=dat.hec_z_km * u.km,
                                    representation_type='cartesian', obstime=DATE_OBS)
soho_hae

<HeliocentricMeanEcliptic Coordinate (equinox=J2000.000, obstime=2008-02-01T20:50:00.000): (x, y, z) in km
    (-97794703.91953, 1.08147814e+08, -112712.32332)>

In [8]:
dat.gci_x_km, dat.gci_y_km, dat.gci_z_km

(1337759.22141, -824922.22264, -479016.13027)

In [9]:
soho_hae.transform_to(GeocentricEarthEquatorial(obstime=DATE_OBS)).cartesian

<CartesianRepresentation (x, y, z) in km
    (1337761.60758917, -824907.72384285, -479041.75376757)>

### Note that GCI in `.DAT` files and `GeocentricEarthEquatorial` coordinates are equivalent as well as `HEC` and `HeliocentricMeanEcliptic` coordinates

In [10]:
soho_hae.transform_to(GeocentricSolarEcliptic(obstime=DATE_OBS)).cartesian

<CartesianRepresentation (x, y, z) in km
    (1600806.90329924, 352943.08061069, -111392.61050122)>

In [11]:
dat.gse_x_km, dat.gse_y_km, dat.gse_z_km

(1600807.58574, 352939.24964, -111363.46095)

### Note that GCI in `.DAT` files and `GeocentricSolarEcliptic` coordinates

In [12]:
soho_hae.transform_to(GeocentricSolarMagnetospheric(obstime=DATE_OBS)).cartesian

<CartesianRepresentation (x, y, z) in km
    (1600805.916001, 367029.48640008, 47640.83182523)>

In [13]:
dat.gsm_x_km, dat.gsm_y_km, dat.gsm_z_km

(1600807.58574, 366972.47487, 47949.31764)

### Note that GSM in `.DAT` files and `GeocentricSolarMagnetic` coordinates

In [14]:
soho = get_horizons_coord('SOHO', DATE_OBS)

INFO: Obtained JPL HORIZONS location for SOHO (spacecraft) (-21) [sunpy.coordinates.ephemeris]


In [15]:
soho.transform_to(GeocentricSolarMagnetospheric(obstime=DATE_OBS)).cartesian.get_xyz().to(u.km)

<Quantity [1600805.86387881,  367013.45605847,   47665.36805124] km>

### Note that `.DAT` agrees with JPL HORIZONS

In [16]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS))

<HeliographicCarrington Coordinate (obstime=2008-02-01T20:50:00.000, rsun=695700.0 km, observer=self): (lon, lat, radius) in (deg, deg, km)
    (261.38005539, -6.07627455, 1.45807292e+08)>

In [17]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).cartesian

<CartesianRepresentation (x, y, z) in km
    (-21730750.76811964, -1.4335038e+08, -15434040.09594443)>

In [18]:
soho.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).cartesian.get_xyz().to(u.km)

<Quantity [-2.17307451e+07, -1.43350384e+08, -1.54340116e+07] km>

In [19]:
soho.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS))

<SkyCoord (HeliographicCarrington: obstime=2008-02-01T20:50:00.000, rsun=695700.0 km, observer=self): (lon, lat, radius) in (deg, deg, AU)
    (261.38005785, -6.0762633, 0.97466155)>

In [20]:
soho.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).radius.to(u.km)

<Distance 1.45807292e+08 km>

In [21]:
soho.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).lon.value

261.38005784960797

In [22]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).lon.value

261.3800553865539

In [23]:
soho.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).lat.value

-6.07626329936262

In [24]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).lat.value

-6.076274552683176

### Debug output from `build_subA.c`

```
fitstime = 20:56:38.096
fitsdate = 2008/02/01
get_orbit.c: sdate is: 2008-02-01.8722, modified julian date is: 54497.872
get_orbit.c: sdate is: 2008-02-01.8722, modified julian date is: 54497.872
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V00.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V01.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V02.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V03.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V04.DAT
local orbit file retrieved: /Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080201_V04.DAT
Computed dist: 209.5837401 Rsun
Computed dsun: 209.5837261 Rsun
Header's dsun: 3.01640576e-07 Rsun

            Sun_ob1: [-140.59, 142.673, 61.6796]
      Rz(a) Sun_ob1: [200.302, 1.42109e-14, 61.6796]
Ry(b) Rz(a) Sun_ob1: [209.584, 1.42109e-14, 7.10543e-15]
        R12 Sun_ob1: [209.584, 1.24345e-14, 1.42109e-14]
              spol2: [-0.105861, -8.32667e-17, 0.994381]
              spol3: [-8.32667e-17, 1.38778e-17, 1]
Polarized Brightness image.
Polar angle: -0.10606 radians = -6.07677 deg
     Header's Observed Latitude = 2.54639e-313 deg
Carrington longitude: -1.7260937832 radians =  -98.8978888 deg
COMPUTED sun_ob1:        [-140.5895916, 142.6728037, 61.67962422]
HEADER'S J2000 sun_obs:  [nan, nan, nan]
      Computed sun_ob3:  [-32.23499602, -205.8980153, -22.18672021]

Sub-Spacecraft Latitude  computed as ATAN(sun_ob3[2]/Sqrt{sun_ob3[0]^2+sun_ob3[1]^2)}: -6.076768492 deg
Sub-Spacecraft Longitude computed as ATAN{sun_ob3[1]/sun_ob3[0]}:                       261.1021112 deg
```


```
fitstime = 20:56:38.136
fitsdate = 2008/05/09
get_orbit.c: sdate is: 2008-05-09.8722, modified julian date is: 54595.872
get_orbit.c: sdate is: 2008-05-09.8722, modified julian date is: 54595.872
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V00.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V01.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V02.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V03.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V04.DAT
local orbit file retrieved: /Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080509_V04.DAT
Computed dist: 215.3094725 Rsun
Computed dsun: 215.3094763 Rsun
Header's dsun: 3.092648849e-07 Rsun

            Sun_ob1: [-139.694, -150.379, -65.0369]
      Rz(a) Sun_ob1: [205.252, -2.84217e-14, -65.0369]
Ry(b) Rz(a) Sun_ob1: [215.309, -2.84217e-14, 0]
        R12 Sun_ob1: [215.309, -2.84217e-14, -1.42109e-14]
              spol2: [-0.0550872, 5.55112e-17, 0.998482]
              spol3: [-4.16334e-17, 3.46945e-17, 1]
Polarized Brightness image.
Polar angle: -0.0551151 radians = -3.15786 deg
     Header's Observed Latitude = 2.54639e-313 deg
Carrington longitude: 0.855213094513 radians =  49.0001009 deg
COMPUTED sun_ob1:        [-139.6941935, -150.3791894, -65.03693035]
HEADER'S J2000 sun_obs:  [nan, nan, nan]
      Computed sun_ob3:  [141.0409499, 162.2496299, -11.86080313]

Sub-Spacecraft Latitude  computed as ATAN(sun_ob3[2]/Sqrt{sun_ob3[0]^2+sun_ob3[1]^2)}: -3.157864433 deg
Sub-Spacecraft Longitude computed as ATAN{sun_ob3[1]/sun_ob3[0]}:                       409.0001009 deg
```

```
fitstime = 14:56:38.330
fitsdate = 2008/03/23
get_orbit.c: sdate is: 2008-03-23.6222, modified julian date is: 54548.622
get_orbit.c: sdate is: 2008-03-23.6222, modified julian date is: 54548.622
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V00.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V01.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V02.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V03.DAT
orbfn =/Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V04.DAT
local orbit file retrieved: /Users/butala/src/solar/prelim/tomroot/Orbits/SO_OR_PRE_20080323_V04.DAT
Computed dist: 212.2365387 Rsun
Computed dsun: 212.2365475 Rsun
Header's dsun: 3.053169653e-07 Rsun

            Sun_ob1: [-211.844, -11.8, -5.21511]
      Rz(a) Sun_ob1: [212.172, 3.55271e-14, -5.21511]
Ry(b) Rz(a) Sun_ob1: [212.237, 3.55271e-14, 0]
        R12 Sun_ob1: [212.237, 3.28626e-14, 1.5099e-14]
              spol2: [-0.120666, -5.55112e-17, 0.992693]
              spol3: [-4.16334e-17, -4.16334e-17, 1]
Polarized Brightness image.
Polar angle: -0.120961 radians = -6.93054 deg
     Header's Observed Latitude = 2.54639e-313 deg
Carrington longitude: -0.818664245831 radians =  -46.9060061 deg
COMPUTED sun_ob1:        [-211.844082, -11.79998733, -5.215105099]
HEADER'S J2000 sun_obs:  [nan, nan, nan]
      Computed sun_ob3:  [143.939935, -153.8498918, -25.60972451]

Sub-Spacecraft Latitude  computed as ATAN(sun_ob3[2]/Sqrt{sun_ob3[0]^2+sun_ob3[1]^2)}: -6.930537962 deg
Sub-Spacecraft Longitude computed as ATAN{sun_ob3[1]/sun_ob3[0]}:                       313.0939939 deg
```

In [25]:
np.array([-140.59, 142.673, 61.6796]) * 6.957e5
#np.array([-139.694, -150.379, -65.0369]) * 6.957e5
#np.array([-211.844, -11.8, -5.21511]) * 6.957e5

array([-97808463.  ,  99257606.1 ,  42910497.72])

In [26]:
soho_hae.transform_to(HCRS()).cartesian

<CartesianRepresentation (x, y, z) in km
    (-96734336.88844943, 1.00350532e+08, 43339784.07194095)>

In [27]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS)).cartesian

<CartesianRepresentation (x, y, z) in km
    (-21730750.76811964, -1.4335038e+08, -15434040.09594443)>

In [28]:
np.array([-32.23499602, -205.8980153, -22.18672021]) * 6.957e5
#np.array([141.0409499, 162.2496299, -11.86080313]) * 6.957e5
#np.array([143.939935, -153.8498918, -25.60972451]) * 6.957e5

array([-2.24258867e+07, -1.43243249e+08, -1.54353013e+07])

In [29]:
soho_hae.transform_to(HeliographicCarrington(observer='self', obstime=DATE_OBS))

<HeliographicCarrington Coordinate (obstime=2008-02-01T20:50:00.000, rsun=695700.0 km, observer=self): (lon, lat, radius) in (deg, deg, km)
    (261.38005539, -6.07627455, 1.45807292e+08)>

In [30]:
HeliographicCarrington(x=-2.24258867e+07*u.km, y=-1.43243249e+08*u.km, z=-1.54353013e+07*u.km, representation_type='cartesian')
#HeliographicCarrington(x=9.81221888e+07*u.km, y=1.12877068e+08*u.km, z=-8.25156074e+06*u.km, representation_type='cartesian')
#HeliographicCarrington(x=1.00139013e+08*u.km, y=-1.07033370e+08*u.km, z=-1.78166853e+07*u.km, representation_type='cartesian')

<HeliographicCarrington Coordinate (obstime=None, rsun=695700.0 km, observer=None): (x, y, z) in km
    (-22425886.7, -1.43243249e+08, -15435301.3)>

In [31]:
HeliographicCarrington(x=-2.24258867e+07*u.km, y=-1.43243249e+08*u.km, z=-1.54353013e+07*u.km, representation_type='cartesian').spherical
#HeliographicCarrington(x=9.81221888e+07*u.km, y=1.12877068e+08*u.km, z=-8.25156074e+06*u.km, representation_type='cartesian').spherical
#HeliographicCarrington(x=1.00139013e+08*u.km, y=-1.07033370e+08*u.km, z=-1.78166853e+07*u.km, representation_type='cartesian').spherical

<SphericalRepresentation (lon, lat, distance) in (deg, deg, km)
    (261.10211118, -6.07676852, 1.45807398e+08)>